In [1]:
#Importing libraries for data management
import pandas as pd
import numpy as np

#Importing libraries for system management
import os

#Distance computation
from pyjarowinkler import distance
import Levenshtein

In [2]:
#TODO: Check if Danish computers have the same problem...
I_want_to_run_all_the_preprocessing = False

# Converting the current format (ISO-8859-14) into UTF-8 
iconv -f ISO-8859-14 file_in.csv > file_out.tsv

In [3]:

if I_want_to_run_all_the_preprocessing:
    
    #Copying into a new directory
    ! cp -R ../../city_dump/data/LL/ ../../city_dump/data/utf8/

    #Getting all files
    d_path = '../../city_dump/data/utf8/'
    l = next(os.walk(d_path))[2]

    #Converting the files to utf8 (not sure this will work in computers different than mac...)
    for file in l:
        f_path = d_path+file
        tmp_path = d_path+'tmp.csv'
        ! iconv -f ISO-8859-14 $f_path > $tmp_path
        !mv $tmp_path $f_path

# Reading DigDag
and transforming it so I encode as many logical variation of the names as possible. In this way the variations for each place is only appied once whereas if we do the same for the data it will need to be applied to each place taking much longer. But the way to remember the original variation is to keep the "enhedid", which is the reference id.

In [4]:
# This function adds extra rows in the dataframe *dd* for the 
# places where *word* exists and replaces it with *replacement*
def adding_extra_rows(dd, word, replacement):
    dd['special'] = dd['simplename'].apply(lambda x: word in x) 
    dd['simplename2'] = dd.simplename.apply(lambda x: x.replace(word,replacement))
    return pd.concat([dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename']],dd[dd['special']][['navn', 'enhedid', 'enhedtype', 'art', 'simplename2']].rename(columns={'simplename2':'simplename'})])

In [5]:
#The digdag seems to have duplicates which I do not understand why
dd = pd.read_csv('../../city_dump/data/rl_places_digdag_v1.txt', sep = '\t', encoding='utf-16', dtype=str)
dd_org = dd.copy()

#Getting the Købstad and putting it to the original data
dd2 = pd.read_csv('../../city_dump/data/koebstad.csv', sep = ';', encoding='utf-8', dtype=str)
dd2['enhedtype'] = dd2['art'] 
dd = pd.concat([dd, dd2[~dd2.isna()][['navn','enhedid','enhedtype','art','simplename']]])
dd['simplename'] = dd['simplename'].astype(str)


#Moving the Købstad to the right column
dd['art'] = np.where(dd['enhedtype'] == 'Købstad', ['Købstadskommune']*len(dd), dd['art'])

#adding a conversion of special characters to latin letters (å -> aa, ø -> oe, æ -> ae) and adding them to the reference list
dd = adding_extra_rows(dd, 'å', 'aa')
dd = adding_extra_rows(dd, 'ø', 'oe')
dd = adding_extra_rows(dd, 'æ', 'ae')

#Preparing the names to append them at the simple name
dd['art'] = dd['art'].apply({'Amt':'amt', 'Sogn':'sogn','Købstadskommune':'købstad', 'Geografisk Herred':'herred', 'Processing':''}.get)

#Adding a duplicated list of their unit type to increase matches
dd['simplename2'] = dd.apply(lambda row: str(row['simplename'])+' '+str(row['art']), axis=1)
dd = pd.concat([dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename']],dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename2']].rename(columns={'simplename2':'simplename'})])

#Adding the plural købstæder as well
dd = adding_extra_rows(dd, 'købstad', 'købstæder')

#Adding spaces instead of hyphens (new rows)
dd = adding_extra_rows(dd, '-', ' ')

#Removing spaces (new rows)
dd = adding_extra_rows(dd, ' ', '')

#Adding bysogn and landsogn (new rows)
dd = adding_extra_rows(dd, 'sogn', 'bysogn')
dd = adding_extra_rows(dd, 'sogn', 'landsogn')

#adding rows where the last letter's "e" will be removed
dd['special'] = dd['simplename'].str[-1] == 'e'
dd['simplename2'] = dd['simplename'].str[:-1]
dd = pd.concat([dd[['navn', 'enhedid', 'enhedtype', 'art', 'simplename']],dd[dd['special']][['navn', 'enhedid', 'enhedtype', 'art', 'simplename2']].rename(columns={'simplename2':'simplename'})])

#Dropping duplicates from digdag
#print(dd[dd.duplicated('simplename', keep=False)].sort_values('simplename').head(30))
dd.drop_duplicates(['art','simplename'], keep='first', inplace=True)

#Drop 'Non' from the list, this may be one of my artifacts...
dd = dd[(dd.simplename.apply(lambda x: not 'Non' in x))]

# TODO take into consideration that the same city name can be in varios geographical areas...
# It should be matched with herred to increase accuracy for each record. However, right now we're only cleaning
# not sure how pressing the issue is...
nr_sogn = len(dd_org[dd_org.art=='Sogn'].simplename.unique()) 
dd.head(20)

#TODO: Right now I can map things to amt for example which should not be the case...
#dd =dd[dd.art='sogn']

,navn,enhedid,enhedtype,art,simplename
0,Kronborg Amt,118765,11,amt,kronborg
1,Præstø Amt,118791,11,amt,præstø
2,Bornholms Amt,118792,11,amt,bornholms
3,Svendborg Amt,118813,11,amt,svendborg
4,Ålborg Amt,118819,11,amt,ålborg
5,Sorø Amt,118785,11,amt,sorø
6,Århus Amt,118846,11,amt,århus
7,Hjørring Amt,118820,11,amt,hjørring
8,Bøvling Amt,118851,11,amt,bøvling
9,Vejle Amt,118849,11,amt,vejle


In [6]:
print('length original:\t', len(dd_org), '\nlength modified:\t', len(dd), '\nunique sogn keys:\t', nr_sogn)

length original:	 2519 
length modified:	 27097 
unique sogn keys:	 1924


# Processing functions

In [7]:
#Get the df from the $ separated file (for some reason pandas has problems with random lines)
def get_df(f_path):
    r= []
    columns = []
    with open(f_path) as f:
        first = True
        for line in f:
            line = line.rstrip().split('$')
            if first:
                length = len(line)
                columns = line
                first=False
            else:
                r.append(line[:length])
                
    return pd.DataFrame(data=r, columns = columns)

In [8]:
# Name cleaning function
def name_cleaner(s, working_column):
    try:
        o =s[working_column].lower().rstrip().replace('  ', ' ')#.replace(' købstad', '')
        if ' (' in o: return o.split(' (')[0]
        return o
    except:
        return np.nan

# Grouping all 1901 files together

In [9]:
if I_want_to_run_all_the_preprocessing:
    
    l =['ft1901_LL_aalborg.txt',
     'ft1901_LL_aarhus.txt',
     'ft1901_LL_bornholm.txt',
     'ft1901_LL_frederiksborg.txt',
     'ft1901_LL_hjoerring.txt',
     'ft1901_LL_holbaek.txt',
     'ft1901_LL_kbhv.txt',
     'ft1901_LL_maribo.txt',
     'ft1901_LL_odense.txt',
     'ft1901_LL_praestoe.txt',
     'ft1901_LL_randers.txt',
     'ft1901_LL_ribe.txt',
     'ft1901_LL_ringkoebing.txt',
     'ft1901_LL_roskilde.txt',
     'ft1901_LL_skanderborg.txt',
     'ft1901_LL_soroe.txt',
     'ft1901_LL_svendborg.txt',
     'ft1901_LL_thisted.txt',
     'ft1901_LL_vejle.txt',
     'ft1901_LL_viborg.txt']

    _path = '../../city_dump/data/utf8/'

    df_list = []
    for f in l:

        print(f)
        #Loading the data
        df_list.append(get_df(_path+f))

    #Concatenating all the files
    df_list = pd.concat(df_list, sort=False)

    #Saving the unique file
    df_list.to_csv(_path+'ft1901_LL.txt', sep='$', index=False)

In [10]:
# List of files to work (with the 1901 collapsed)
working_data = ['ft1845_LL.txt',
 'ft1850_LL.txt',
 'ft1860_LL.txt',
 'ft1880_LL.txt',
 'ft1885_LL.txt',
 'ft1901_LL.txt']

# Loading data and counting

In [11]:
_path = '../../city_dump/data/utf8/'

#This is where I save the counts and then group it together
out_list = []

#where I save the missmatches and file year
missmatches = []
total_records = 0

for f in working_data:
    
    print(f)
    #Loading the data
    df = get_df(_path+f)
    
    #Dropping empty rows due to the "mal-conversion" to utf8... The empty rows do contain data originally
    #TODO: Fix the conversion thing (ask Barbara to provide me the UTF-8...)
    df = df[~df.Herred.isna()]
    print(len(df))
    total_records = total_records + len(df)
    
    #Preforming the name cleaning for Sogne, Herred, and Amt
    for working_column in ['Sogne','Herred','Amt']:
        df[working_column+'_data'] = df.apply(lambda x: name_cleaner(x, working_column), axis=1)
    
    #Reducing Dimensionality and saving in RAM
    out = df.groupby('Sogne_data').size().reset_index(name = 'counts')
    out['year'] = f[2:6]
    out_list.append(out)
    
out_list = pd.concat(out_list, sort=False)
print('Done! :D')

out_list.head()

ft1845_LL.txt
1489875
ft1850_LL.txt
1391708
ft1860_LL.txt
1715906
ft1880_LL.txt
1967615
ft1885_LL.txt
327936
ft1901_LL.txt
1979025
Done! :D


,Sogne_data,counts,year
0,,1,1845
1,aabenraa landsogn,444,1845
2,aaby,1520,1845
3,aadum,554,1845
4,aagerup,1046,1845


In [13]:
#Putting it nicely, on a more horitzontal table
out_list = out_list.pivot('Sogne_data', 'year', 'counts').fillna(0).astype(int).reset_index().reset_index(drop=True)
out_list = pd.DataFrame(out_list.values, columns=out_list.columns.tolist()) #Getting rid of the wrong index
out_list.head()

,Sogne_data,1845,1850,1860,1880,1885,1901
0,,1,0,0,0,0,0
1,(h. c.) andersensvej,0,0,0,744,0,0
2,a. f. beyersvej,0,0,0,0,0,78
3,a. n. hansens allé,0,0,0,0,0,10
4,aabenraa,0,0,0,1404,1314,1044


# Basic matching
(only performs the match if the names are identical)

_This could be done only once by countring reference id and year. Then with the names we can keep constraining with matches_

In [14]:
#Notice that I only match for Sogn
out = out_list.merge(dd[['navn','simplename', 'art', 'enhedid']], left_on='Sogne_data', right_on='simplename', how='left')

#Those are the missmatch
miss = out[out.enhedid.isna()]
miss.head()

#Here are the match
match = out[~out.enhedid.isna()]
match.head()

# Aggregating the match by enhedid and year
agg_counts = match.groupby(['navn', 'enhedid', 'art'],as_index = False).agg({'1845':'sum', '1850':'sum','1860':'sum', '1880':'sum','1885':'sum', '1901':'sum'})

# Grouping all enhedid to see how many different versions of the same Sogne we have in the data
diff_counts = match.groupby(['navn','enhedid'])['Sogne_data'].apply(set).reset_index(name='extended_digdag') #if parenthesis is confusing add **.apply(lambda x: ', '.join(x))** before the reset index

# Joining DFs and saving
s = agg_counts.merge(diff_counts, on = ['navn','enhedid'])
s.to_csv(_path+'../out/places_FT_uniquevalues_01.tsv', sep='\t', index=False)
print(len(s))
s.head()

1741


,navn,enhedid,art,1845,1850,1860,1880,1885,1901,extended_digdag
0,Aabenraa Sogn,113932,sogn,444,0,650,1404,1314,1044,"{aabenraa, aabenraa landsogn}"
1,Aaker Sogn,115247,sogn,1864,1956,1978,2485,0,2421,{aaker}
2,Aal Sogn,115248,sogn,917,972,962,1097,0,1299,{aal}
3,Aarestrup Sogn,115255,sogn,468,523,611,773,0,844,{aarestrup}
4,Abild Sogn,113795,sogn,90,0,1165,0,0,0,{abild}


In [15]:
#TODO: Compute measure metrics
#Something like
'''
print('Matched records:',nr_matched_records, 'out of', total_records, '(', nr_matched_records/total_records*100,')%')
print('Matched places:', len(out_list), 'out of', nr_sogn, '(', len(out_list)/nr_sogn*100,')%')
''';

# Computing Jaro distances for missmatches

In [16]:
#Getting the potential first matches

mm = miss.Sogne_data.unique()
#sn = dd_org.simplename.unique() # It's a smaller list, and the extended version adds "meaningless info": sogne, landsogn, etc. which may distract the algorithm
sn = dd.simplename.unique() # this uses my extended version of digdag

distance_jaro = np.zeros((len(sn),len(mm)))
distance_leven = np.zeros((len(sn),len(mm)))

for i in range(len(sn)):
    if i%100 == 0: print(i, 'out of' , len(sn))
        
    for j in range(len(mm)): #The internal loop could be done using apply (it should be the longer one)
        #If variable is none skip
        if not mm[j]: continue
        
        #This matrix is not simetric because x and y axis are not the same!
        try:
            distance_jaro[i][j] = distance.get_jaro_distance(sn[i],mm[j])
            distance_leven[i][j] = Levenshtein.distance(sn[i],mm[j])/(len(sn[i])+len(mm[j]))
        except:
            print('Could not make it',i,j)
            print(sn[i],mm[j])
        
#Index is the reference names found in digdag the columns the original data
distance_jaro = pd.DataFrame(data = distance_jaro, columns = mm, index = sn)
distance_leven = pd.DataFrame(data = distance_leven, columns = mm, index = sn)
print(distance_jaro.shape, 'len mm,sn:', len(mm), len(sn)) #(26428, 1237)
distance_jaro.head()

0 out of 26784
100 out of 26784
200 out of 26784
300 out of 26784
400 out of 26784
500 out of 26784
600 out of 26784
700 out of 26784
800 out of 26784
900 out of 26784
1000 out of 26784
1100 out of 26784
1200 out of 26784
1300 out of 26784
1400 out of 26784
1500 out of 26784
1600 out of 26784
1700 out of 26784
1800 out of 26784
1900 out of 26784
2000 out of 26784
2100 out of 26784
2200 out of 26784
2300 out of 26784
2400 out of 26784
2500 out of 26784
2600 out of 26784
2700 out of 26784
2800 out of 26784
2900 out of 26784
3000 out of 26784
3100 out of 26784
3200 out of 26784
3300 out of 26784
3400 out of 26784
3500 out of 26784
3600 out of 26784
3700 out of 26784
3800 out of 26784
3900 out of 26784
4000 out of 26784
4100 out of 26784
4200 out of 26784
4300 out of 26784
4400 out of 26784
4500 out of 26784
4600 out of 26784
4700 out of 26784
4800 out of 26784
4900 out of 26784
5000 out of 26784
5100 out of 26784
5200 out of 26784
5300 out of 26784
5400 out of 26784
5500 out of 26784
5600

,,(h. c.) andersensvej,a. f. beyersvej,a. n. hansens allé,aadum,aagade,aalborg frue landsogn,aalborggade,aalekistevej,aarhus havn skibe i havnen,...,øster bølle,øster farimagsgade,øster kvarter,øster søgade,øster voldgade,østerbrogade,østergade,østersøgade,østervoldgade,østre møllesti
kronborg,0.0,0.00,0.46,0.39,0.00,0.00,0.49,0.54,0.40,0.39,...,0.48,0.45,0.47,0.47,0.53,0.61,0.49,0.48,0.54,0.40
præstø,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.47,...,0.59,0.44,0.47,0.58,0.46,0.47,0.50,0.59,0.47,0.57
bornholms,0.0,0.39,0.39,0.50,0.00,0.00,0.38,0.44,0.46,0.48,...,0.47,0.44,0.40,0.46,0.52,0.46,0.41,0.47,0.52,0.49
svendborg,0.0,0.00,0.45,0.39,0.44,0.35,0.54,0.60,0.40,0.00,...,0.52,0.56,0.52,0.59,0.57,0.67,0.64,0.60,0.68,0.40
ålborg,0.0,0.00,0.00,0.00,0.00,0.44,0.69,0.76,0.42,0.40,...,0.42,0.41,0.41,0.50,0.41,0.47,0.52,0.51,0.33,0.41


## Computing the best match for each data_sogn with at least with a minimum value

In [17]:
THRESHOLD = 0.9

#Getting the highest match for each Sogne and filtering threshold
a = distance_jaro.idxmax().reset_index(name = 'potential_match')
b = distance_jaro.max().reset_index(name='jaro')
dfmap = a.merge(b, on='index').rename(columns={'index':'data_name'})
dfmap.to_csv(_path+'../out/mapping.tsv', sep='\t', index=False)
dfmap = dfmap[dfmap.jaro.astype(float) >= THRESHOLD]
dfmap.head()

#Getting the mapped info into DigDag v2 (not really the v2)
dd2 = dfmap.merge(dd[['navn','simplename', 'art', 'enhedid']].sort_values('art', ascending=False).drop_duplicates('simplename', keep='first'), left_on='potential_match', right_on='simplename')
dd2['simplename'] = dd2['data_name']
dd2 = dd2[['navn','enhedid','art','simplename']]
dd2 = pd.concat([dd2[['navn','simplename', 'art', 'enhedid']], dd], sort=False) #putting together
dd2.head()

,navn,simplename,art,enhedid,enhedtype
0,Fåborg Sogn,aalborg frue landsogn,sogn,113312,NaN
1,Ålborg Købstadskommune,aalborggade,købstad,120656,NaN
2,Århus Købstad,aarhus mark,købstad,120696,NaN
3,Århusgård Amt,aarhusgade,amt,118831,NaN
4,Abildgård Sogn,abildgaardsgade,sogn,113934,NaN


In [18]:
# Starting again with the merges
out = out_list.merge(dd2, left_on='Sogne_data', right_on='simplename', how='left')

#Those are the missmatch
miss = out[out.enhedid.isna()]
miss.head()

#Here are the match
match = out[~out.enhedid.isna()]
match.head()

# Aggregating the match by enhedid and year
agg_counts = match.groupby(['navn', 'enhedid', 'art'],as_index = False).agg({'1845':'sum', '1850':'sum','1860':'sum', '1880':'sum','1885':'sum', '1901':'sum'})

# Grouping all enhedid to see how many different versions of the same Sogne we have in the data
diff_counts = match.groupby(['navn','enhedid'])['Sogne_data'].apply(set).reset_index(name='extended_digdag') #if parenthesis is confusing add **.apply(lambda x: ', '.join(x))** before the reset index

# Joining DFs and saving
s = agg_counts.merge(diff_counts, on = ['navn','enhedid'])
s.to_csv(_path+'../out/places_FT_jaro0.9_01.tsv', sep='\t', index=False)
print(len(s))
s.head()

1835


,navn,enhedid,art,1845,1850,1860,1880,1885,1901,extended_digdag
0,Aabenraa Sogn,113932,sogn,444,0,650,1404,1314,1044,"{aabenraa, aabenraa landsogn}"
1,Aaker Sogn,115247,sogn,1864,1956,1978,2485,0,2421,{aaker}
2,Aal Sogn,115248,sogn,917,972,962,1097,0,1299,{aal}
3,Aarestrup Sogn,115255,sogn,468,523,611,773,0,844,{aarestrup}
4,Abild Sogn,113795,sogn,90,0,1165,0,0,0,{abild}


# Getting counts for other possible matches

In [21]:
# Get the best possible matches

concatenate = []
i=0
for col in distance_jaro.columns:
    i = i+1
    if i % 100 == 0: print(i, 'out of', len(distance_jaro.columns))
    aux = distance_jaro.nlargest(5, col)[[col]]
    aux.columns = ['score']
    aux['original'] = col
    concatenate.append(aux)

concatenate = pd.concat(concatenate, sort=False)
concatenate = concatenate.reset_index()
concatenate.columns = ['potential_match','score','original']
concatenate[['original','potential_match','score']][~concatenate.original.isin(concatenate[(concatenate.score >= 0.9) & ~concatenate.original.isna()].original.unique())].to_csv(_path+'../out/places_possible_matches_jaro_01.tsv', index=False, sep='\t')

concatenate = pd.read_csv(_path+'../out/places_possible_matches_jaro_01.tsv',  sep='\t')
concatenate.head(15)

,original,potential_match,score
0,NaN,kronborg,0.00
1,NaN,præstø,0.00
2,NaN,bornholms,0.00
3,NaN,svendborg,0.00
4,NaN,ålborg,0.00
5,(h. c.) andersensvej,branderslevsogn,0.69
6,(h. c.) andersensvej,gammel haderslev sogn,0.68
7,(h. c.) andersensvej,broenderslev sogn,0.67
8,(h. c.) andersensvej,store arden sogn,0.66
9,(h. c.) andersensvej,gammel haderslev bysogn,0.66


In [22]:
# Getting the digDag info for the matches but also for the non matches

m = concatenate.merge(dd2[['navn','simplename', 'art', 'enhedid']].sort_values('art', ascending=False).drop_duplicates('simplename',keep='first'), left_on='potential_match', right_on='simplename', how='inner')
m = m[['original','potential_match','score','navn','art','enhedid', 'simplename']]

w = dd2.sort_values('art', ascending=False).drop_duplicates('simplename',keep='first')
w['potential_match'] = w['simplename']
w['original'] = w['simplename']
w['score'] = 1

m = pd.concat([w, m], sort=True)


#Putting the things into place
m['original'] = np.where(m.original.isna(), m['simplename'], m['original'])

print(len(concatenate), len(m))
m.head()

4460 31510


,art,enhedid,enhedtype,navn,original,potential_match,score,simplename
0,sogn,113312,NaN,Fåborg Sogn,aalborg frue landsogn,aalborg frue landsogn,1.0,aalborg frue landsogn
183,sogn,114286,76,Måløv Sogn,måloev landsogn,måloev landsogn,1.0,måloev landsogn
180,sogn,114589,76,Sjælør Sogn,sjæloer landsogn,sjæloer landsogn,1.0,sjæloer landsogn
179,sogn,114282,76,Mørke Sogn,moerke landsogn,moerke landsogn,1.0,moerke landsogn
175,sogn,114277,76,Møborg Sogn,moeborg landsogn,moeborg landsogn,1.0,moeborg landsogn


In [23]:
#Getting the counts for the original plus the counts for the potential matches in DigDag
#Warning: Potential matches may not exist in the data...
out = m.merge(out_list, left_on='original', right_on='Sogne_data', how='left').merge(out_list, left_on='potential_match', right_on='Sogne_data', how='left').fillna(0)
for y in ['1845', '1850', '1860', '1880', '1885', '1901']:
    out[y] = out[y+'_x'] + out[y+'_y']
out = out[['original', 'potential_match', 'score', 'navn', 'art', 'enhedid', '1845', '1850', '1860', '1880', '1885', '1901']].sort_values('original')
out.to_csv(_path+'../out/places_possible_matches_jaro_01.tsv', sep='\t', index=False)

In [24]:
print('Done! :D')

Done! :D
